## 超参数

In [1]:
import numpy as np
from sklearn import datasets

In [3]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [7]:
from sklearn.neighbors import KNeighborsClassifier
 
knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
# knn_clf.predict(X_test)
knn_clf.score(X_test, y_test)

0.9888888888888889

In [8]:
'''
n_neighbors 之前我们都是随意传 3, 5, 6

但是究竟传什么才是最好的呢?(后面会用 超参数GridSearchCV网格搜索)

 超参数 就是运行机器学习算法之前需要指定的参数
 
 注意这门课不是为了从底层设计机器学习算法库, 而是为了更好地使用官方的机器学习算法库



'''

'\nn_neighbors 之前我们都是随意传 3, 5, 6\n\n但是究竟传什么才是最好的呢?(后面会用 超参数GridSearchCV网格搜索)\n\n 超参数 就是运行机器学习算法之前需要指定的参数\n\n\n'

### 寻找最好的k

In [13]:
'''
best_score: 最好准确率的值初始化为0
best_k = -1

通过循环的形式来寻找最好的k

调参过程
'''
best_score = 0.0
best_k = -1

for k in range(1, 11):
    knn_clf = KNeighborsClassifier(n_neighbors=k)
    knn_clf.fit(X_train, y_train)
    score = knn_clf.score(X_test, y_test)
    if score > best_score:
        best_k = k
        best_score = score

print("best_k = ", best_k)
print("best_score = ", best_score)

best_k =  4
best_score =  0.9916666666666667


In [12]:
'''
上面我们循环只进行到了10

我们可以试试 8~20 的范围有没有可能得到更好的结果
'''
# best_score = 0.0
# best_k = -1

# for k in range(8, 21):
#     knn_clf = KNeighborsClassifier(n_neighbors=k)
#     knn_clf.fit(X_train, y_train)
#     score = knn_clf.score(X_test, y_test)
#     if score > best_score:
#         best_k = k
#         best_score = score

# print("best_k = ", best_k)
# print("best_score = ", best_score)

best_k =  8
best_score =  0.9861111111111112


In [14]:
'''
注意: 不要以为 kNN 中只有 k 一个超参数

还有 权值!

'''

'\n注意: 不要以为 kNN 中只有 k 一个超参数\n\n还有 权值!\n\n'

### 考虑距离? 不考虑距离?

使用uniform好呢, 还是distance好呢?

In [16]:
'''
我们这里设置一个新的参数 best_method, 表示我们到底要不要考虑距离的权重, 初始化为空

初始化 method 循环, 再每一种method情况下再来看去不同的值结果怎么样
(在KNeighborsClassifier中传入参数weighs=method)

如果score > best_score, 除了记录之前的best_k, best_score,
还要记录 best_method
'''

best_method = ""
best_score = 0.0
best_k = -1

for method in ['uniform', 'distance']:
    for k in range(1, 11):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights=method)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_score = score
            best_method = method

print("best_method = ", best_method)            
print("best_k = ", best_k)
print("best_score = ", best_score)

best_method =  uniform
best_k =  4
best_score =  0.9916666666666667


<font size=3>我们可以发现, 对于手写数字识别这个例子来说, 我们找到了最佳调用kNN的方法依然是使用我们的uniform方法, 相应的k值取4, 预测分数是0.9916...</font>

<br/>

<font size=3>距离, 什么是距离呢?</font>

to .md

### <font size=3>搜索明可夫斯基距离相应的p</font>

'''
现在我们是对 best_p 进行搜索

外层 k 循环不变, 
里面嵌套一层 p 循环 从 1~5

那么我们 knn_clf 的 weighs='distance', p=p

那么现在我们就是对 k 和 p 这两个超参数进行搜索
'''

In [18]:
%%time

best_p = ""
best_score = 0.0
best_k = -1

for k in range(1, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights='distance', p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_score = score
            best_p = p

print("best_p = ", best_p)            
print("best_k = ", best_k)
print("best_score = ", best_score)

best_p =  2
best_k =  3
best_score =  0.9888888888888889
Wall time: 36.7 s


上面我们求出来了对于识别手写输入文字例子来说, 最好的 p 和 k 是 2 和 3

这一节我们学习了什么是超参数, 即使对于 kNN 这样的算法, 还是具有很多的超参数, 不仅有 k, weighs, p 这3个

那么对于超参数的选择, 我们就可以选择 这种搜索的策略, 来找到适合我们的超参数

事实上, 这种搜索策略有一种专业的名字, 就叫做 网格搜索 GridSearchCV

在上面就是 对 k * p 的网格中的每一个点所代表的数据进行搜索, 来找其中的最好值

上面超参数 weighs 对 p 是有影响的, weighs='distance' 的时候才会有p

也就是超参数之间还会存在 依赖 的关系, 那么我们怎么才能更好的将超参数全部列出来, 运行一遍程序就能找到最好的超参数组合呢?

事实上, scikit-learn 中, 网格搜索封装了一个专门的函数, 我们调用这个网格搜索方法就能更好地实现超参数寻找的过程

关于这个问题, 将在4.6节介绍